In [67]:
import pandas as pd
import altair as alt
from altair.expr import datum
import warnings
warnings.filterwarnings('ignore')

In [68]:
# load mean_attr dataframe
mean_attr_filename = '../Saved/4vis/mean_attr.pkl'
mean_attr_melted = pd.read_pickle(mean_attr_filename)

In [ ]:
dict_method_name_to_attr_name = {
    'shap' : 'shap_value',
    'lime': 'lime_weight', 
    'ig':  'ig_attr', 
    'deepLift':'deepLift_attr' 
}

## Make the Charts

In [85]:
## bar chart for feature attribution values and rank
brush = alt.selection_multi(on='mouseover', toggle=True, fields=['feature_name'])
base = alt.Chart(mean_attr_melted).mark_bar().encode(
    opacity = alt.condition(brush, alt.value(1.0), alt.value(0.2))
).add_selection(
    brush
)

In [86]:
# place the 4 charts of mean abs feature attribution value in a grid
row1 = alt.hconcat()
row2 = alt.hconcat()
i = 0
for _, attr_name in dict_method_name_to_attr_name.items():
    chart = base.encode(
        y=alt.Y('feature_name', sort='-x', title = ''),
        x= alt.X('feature_attr', title = attr_name),
    ).transform_filter(
        datum.method == attr_name
    ).properties(
        width=200,
        height=200
    )
    if i < 2:
        row1 |= chart
    else:
        row2 |= chart
    i += 1
attr_value_bar = alt.vconcat(row1, row2)

In [87]:
# group bar chart visualization for mean attr rank
sorted_feature = mean_attr_melted[mean_attr_melted['method'] == 'sum_rank'].sort_values(by = 'feature_attr', ascending = False)['feature_name'].to_list() 
mean_attr_rank = base.mark_bar(size = 5).encode(
    x = alt.X('method', title=''),
    y = alt.X('feature_attr', title = 'feature_rank'),
).transform_filter(
    alt.FieldOneOfPredicate(field='method', oneOf= ['shap_rank', 'lime_rank', 'ig_rank', 'deepLift_rank'])
).properties(
    width=50,
    height=200
).facet(column=alt.Column('feature_name', sort = sorted_feature)) 

mean_attr_values_and_rank_plot = alt.hconcat(attr_value_bar, mean_attr_rank)

## Show the charts


In [90]:
mean_attr_values_and_rank_plot

alt.HConcatChart(...)